In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import keras.models
import numpy as np

In [11]:
#Fix the random generator seeds for better reproducibility
tf.random.set_seed(67)
np.random.seed(67)

In [21]:
    #Class to build the Model
class Model_builder:
        def __init__(self, input_shape, num_classes):
            self.input_shape = input_shape
            self.num_classes = num_classes
            self.model = None
        def build(self):

        

            input_img = Input(shape=self.input_shape)
            x= input_img
            # AUTOENCODER: how stacked should it be/How many layers? 
            #1st layer
            for _ in range(5):
                x = Conv2D(
                    filters=32,
                    kernel_size=3, #could be changed manually
                    activation='relu',
                    padding='same',
                    activity_regularizer=regularizers.l1(0.0001),
                    kernel_initializer='he_normal'
                )(x)
                x = MaxPooling2D((2, 2), padding='same')(x) #max pooling layer

            #max pooling layer to give us the result of the encoding process: latent space
            encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
            print(encoded.shape)

            flattened = tf.keras.layers.Flatten()(encoded)
            dense_1 = tf.keras.layers.Dense(1792, activation='relu')(flattened)
            dense_2 = tf.keras.layers.Dense(100, activation='relu')(dense_1)
            output = tf.keras.layers.Dense(2, activation='softmax')(dense_2)
                   
                        
                        
                        #start of decoding, remove comment for decoding
                        #x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
                        #x = UpSampling2D((2, 2))(x)
                        #x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
                        #x = UpSampling2D((2, 2))(x)
                        #decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)


            #PRETRAINED EFFICIENTNET, could possibly use smaller but less computationaly expensive models: try,
            # note that use of LSTM is computationally more expensive and is not sure to bring better accuracy but still worth trying 
            #also note the use of tranfer learning here, but could also go with the route of training the whole model: after HP tuning with validation data ofc
            #efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=self.input_shape)
            #for layer in efficientnet.layers[:-20]:  #freezing everything except top 20 layers, again could also go for layer.trainable=true for all layers: do we have enough computational resources?
                #layer.trainable = False
            #x = efficientnet(encoded) #!!!!!!!!!!!!!!!!!!!!!!!change to encoded, change dimensions in other places accordingly
            #x = GlobalAveragePooling2D()(x) #helps reduce overfitting by reducing the total number of parameters in the model
            #x = Dense(32,kernel_initializer='glorot_uniform')(x) #A fully connected dense layer, possibly for feature extracion, could think of adding more layers here for feature classifications in the future
            #x = Dropout(0.3)(x)#for regularization, again use of tuning: maybe too much?
            #output = Dense(self.num_classes, activation='softmax')(x)#fully connected dense layer with softmax activation for producing the output probabilities of classes
                #This works as the classifier in our ML pipeline thanks to softmax


            #create the model using keras Model function with all the earlier configuration and return it
            self.model = tf.keras.models.Model(inputs=input_img, outputs=output)

        
        #compile the model
        def compile(self):
            self.model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy']
            )
            return self.model
            
        

In [22]:
#Create a model_builder object, build and compile the model inside it and retrieve it as "model"
model_builder_object = Model_builder(input_shape=(240, 193, 1) ,num_classes=2)#this part can be replaced with correct input shape: 240x193x1
model_builder_object.build() 
model = model_builder_object.compile()
 


(None, 8, 7, 32)


In [14]:
directory = r"C:\Users\bbaki\Desktop\fhnw\Scripts\radio_sunburst_detector\data"
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(240, 193),
    shuffle=True,
    seed=42, #can change
    validation_split=0.2, #can change
    subset="training",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(240, 193),
    shuffle=True,
    seed=42, #can change
    validation_split=0.2, #can change
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)





Found 100 files belonging to 2 classes.


Using 80 files for training.
Found 100 files belonging to 2 classes.
Using 20 files for validation.


In [15]:
#rescale training and validation data
def rescale(image, label):
    return image/255. , label

train_ds = train_ds.map(rescale)
validation_ds = validation_ds.map(rescale)

In [16]:
#Determine the number of images in train and validation datasets
train_total_size = tf.data.experimental.cardinality(train_ds).numpy()
val_total_size = tf.data.experimental.cardinality(validation_ds).numpy()

print("Training data Image number: ", train_total_size)
print("Validation data Image number: ", val_total_size)

Training data Image number:  3
Validation data Image number:  1


In [17]:
#Shuffle the data for the train test split to 
#dataset = dataset.shuffle(total_size)

#Split into train and test sets
#train_dataset = dataset.take(train_size)
#test_dataset = dataset.skip(train_size)

#Function to split dataset into inputs (x) and labels (y)
#def split_into_inputs_and_labels(image, label):
#    return image, label

#Apply this function to both datasets using map function: Iteratively apply the split_into_inputs_and_labels element to 
#every element in train_dataset and test_dataset which consist of image and label thanks to keras image_dataset_from_directory function
#train_x, train_y = zip(*train_dataset.map(split_into_inputs_and_labels))
#test_x, test_y = zip(*test_dataset.map(split_into_inputs_and_labels))

In [19]:
#Train the model
model.fit(
    train_ds,
    validation_data=(validation_ds),
    epochs=20,
    batch_size=4,
)

Epoch 1/20


3/3 [==============================] - 3s 911ms/step - loss: 20.1197 - accuracy: 0.5500 - val_loss: 21.7307 - val_accuracy: 0.4000
Epoch 2/20
3/3 [==============================] - 3s 880ms/step - loss: 19.1236 - accuracy: 0.5250 - val_loss: 20.7402 - val_accuracy: 0.4000
Epoch 3/20
3/3 [==============================] - 3s 921ms/step - loss: 18.2408 - accuracy: 0.5250 - val_loss: 19.8446 - val_accuracy: 0.4000
Epoch 4/20
3/3 [==============================] - 3s 971ms/step - loss: 17.4617 - accuracy: 0.5250 - val_loss: 19.0682 - val_accuracy: 0.4000
Epoch 5/20
3/3 [==============================] - 3s 932ms/step - loss: 16.7767 - accuracy: 0.5250 - val_loss: 18.3693 - val_accuracy: 0.4000
Epoch 6/20
3/3 [==============================] - 3s 924ms/step - loss: 16.1527 - accuracy: 0.5375 - val_loss: 17.7252 - val_accuracy: 0.5000
Epoch 7/20
3/3 [==============================] - 3s 926ms/step - loss: 15.5830 - accuracy: 0.5375 - val_loss: 17.1309 - val_accuracy: 0.5000
Epoch 8/20
3/3 [=